# HyperGen LoRA Training Quickstart

This notebook demonstrates how to train a LoRA on a diffusion model using HyperGen.

We'll:
1. Install dependencies
2. Download a small dataset from HuggingFace
3. Train a LoRA in just 5 lines of code
4. Generate images with the trained LoRA

**Hardware Requirements:**
- GPU with at least 12GB VRAM (e.g., T4, RTX 3060, or better)
- Or use Google Colab with GPU runtime

## 1. Install Dependencies

In [ ]:
# Install HyperGen and dependencies
!pip install -q torch torchvision diffusers transformers accelerate peft pillow datasets tqdm

# If running locally, install hypergen from source
# !pip install -e ..

# For Colab/remote environments
!pip install -q git+https://github.com/ntegrals/hypergen.git

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. Download Dataset from HuggingFace

We'll use a small Pokemon dataset for this example. It's perfect for quick experimentation!

In [ ]:
from datasets import load_dataset
from pathlib import Path
import os

# Load a small Pokemon dataset from HuggingFace
print("Downloading dataset...")
hf_dataset = load_dataset("lambdalabs/naruto-blip-captions", split="train")

# Take only first 50 images for quick training
hf_dataset = hf_dataset.select(range(min(50, len(hf_dataset))))

print(f"Loaded {len(hf_dataset)} images")
print(f"Sample: {hf_dataset[0]['text']}")

In [ ]:
# Save dataset to local folder in HyperGen format
dataset_dir = Path("./training_data")
dataset_dir.mkdir(exist_ok=True)

print("Preparing dataset...")
for idx, item in enumerate(hf_dataset):
    # Save image
    image_path = dataset_dir / f"image_{idx:04d}.png"
    item["image"].save(image_path)
    
    # Save caption
    caption_path = dataset_dir / f"image_{idx:04d}.txt"
    with open(caption_path, "w") as f:
        f.write(item["text"])

print(f"✓ Dataset saved to {dataset_dir}")
print(f"✓ Total images: {len(list(dataset_dir.glob('*.png')))}")

In [ ]:
# Preview some samples from the dataset
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 4, figsize=(15, 8))
axes = axes.flatten()

for idx in range(8):
    if idx < len(hf_dataset):
        axes[idx].imshow(hf_dataset[idx]["image"])
        axes[idx].set_title(hf_dataset[idx]["text"][:50] + "...", fontsize=8)
        axes[idx].axis("off")

plt.tight_layout()
plt.show()

## 3. Train LoRA - Just 5 Lines!

This is where the magic happens. HyperGen makes it incredibly simple.

In [ ]:
from hypergen import model, dataset

# 1. Load model (using SDXL Turbo for speed)
print("Loading model...")
m = model.load("stabilityai/sdxl-turbo")
m.to("cuda")

# 2. Load dataset
print("\nLoading dataset...")
ds = dataset.load("./training_data")
print(f"Dataset size: {len(ds)} images")

# 3. Train LoRA - that's it!
print("\nStarting training...\n")
lora = m.train_lora(
    ds,
    steps=100,              # Quick training for demo
    learning_rate=1e-4,
    batch_size=1,
    rank=16,                # LoRA rank
    alpha=32,               # LoRA alpha
    resolution=512,         # Training resolution
    output_dir="./lora_checkpoints",
    save_steps=50,          # Save every 50 steps
)

print("\n✓ Training complete!")

## 4. Generate Images with Trained LoRA

Now let's test our trained LoRA by generating some images!

In [ ]:
# Load the trained LoRA weights
from peft import PeftModel

# The LoRA is already in the model from training, but if you want to load from checkpoint:
# checkpoint_path = "./lora_checkpoints/checkpoint-100"
# m.pipeline.unet = PeftModel.from_pretrained(m.pipeline.unet, checkpoint_path)

print("LoRA loaded and ready for inference!")

In [ ]:
# Generate images with the trained LoRA
prompts = [
    "naruto uzumaki standing in a forest",
    "sasuke uchiha with lightning",
    "kakashi hatake reading a book",
    "sakura haruno in action pose",
]

print("Generating images...")
images = []
for prompt in prompts:
    print(f"Prompt: {prompt}")
    image = m.generate(
        prompt,
        num_inference_steps=4,  # SDXL Turbo uses few steps
        guidance_scale=0.0,     # Turbo doesn't use guidance
    )
    images.append(image[0] if isinstance(image, list) else image)

print("✓ Generation complete!")

In [ ]:
# Display generated images
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, (image, prompt) in enumerate(zip(images, prompts)):
    axes[idx].imshow(image)
    axes[idx].set_title(prompt, fontsize=10, wrap=True)
    axes[idx].axis("off")

plt.tight_layout()
plt.savefig("generated_images.png", dpi=150, bbox_inches="tight")
plt.show()

print("✓ Images saved to generated_images.png")

## 5. Compare: Before vs After LoRA

Let's compare generations from the base model vs our LoRA-trained model.

In [ ]:
# Generate without LoRA (base model)
from diffusers import DiffusionPipeline

print("Loading base model for comparison...")
base_model = DiffusionPipeline.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
).to("cuda")

test_prompt = "naruto uzumaki in ninja outfit"

print(f"\nGenerating with base model: '{test_prompt}'")
base_image = base_model(
    test_prompt,
    num_inference_steps=4,
    guidance_scale=0.0,
).images[0]

print(f"Generating with LoRA-trained model: '{test_prompt}'")
lora_image = m.generate(
    test_prompt,
    num_inference_steps=4,
    guidance_scale=0.0,
)
lora_image = lora_image[0] if isinstance(lora_image, list) else lora_image

# Display comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(base_image)
axes[0].set_title("Base Model (No LoRA)", fontsize=12)
axes[0].axis("off")

axes[1].imshow(lora_image)
axes[1].set_title("With Trained LoRA", fontsize=12)
axes[1].axis("off")

plt.suptitle(f"Prompt: '{test_prompt}'", fontsize=14)
plt.tight_layout()
plt.savefig("comparison.png", dpi=150, bbox_inches="tight")
plt.show()

print("✓ Comparison saved to comparison.png")

## Next Steps

Congratulations! You've successfully:
- ✅ Loaded a dataset from HuggingFace
- ✅ Trained a LoRA in just 5 lines of code
- ✅ Generated images with your custom LoRA

### Try These Next:

1. **Use your own images**: Replace the dataset with your own image folder
2. **Train longer**: Increase `steps` to 500-1000 for better results
3. **Try different models**: Use `stabilityai/stable-diffusion-xl-base-1.0` or `black-forest-labs/FLUX.1-dev`
4. **Adjust LoRA parameters**: Experiment with `rank` and `alpha`
5. **Serve with API**: Use `hypergen serve` to deploy your model

### Resources:
- [HyperGen Documentation](https://github.com/ntegrals/hypergen)
- [Examples Directory](../examples/)
- [HuggingFace Diffusers](https://huggingface.co/docs/diffusers)
